Teste para EIS Wishart em Julia.

Modelos baseados no projeto CNPq.

Teste trivariado.

Densidade de observáveis:
$$p(y_t|\Sigma_t)\sim N(0,\Sigma_t)$$

Densidade de transição dos estados:

$$p(\Sigma_t^{-1}|\Sigma_{t-1}^{-1},v,d,C)\sim W(v,S_{t-1})$$

sendo que:

$$S_{t-1}=\frac{1}{v}(C^{1/2})(\Sigma_{t-1})^{d}(C^{1/2})^{\prime}$$

In [1]:
# Pacotes
using Distributions
using PDMats
using PyPlot

# Constantes
const LOG2PI = 1.83787706640934533908193770912475883960723876953125;
const LOG2   = 0.69314718055994528622676398299518041312694549560546875;
const LOGPI  = 1.1447298858494001638774761886452324688434600830078125;

# Parametrização
const K  = 3
const T  = 240
const N  = 500
const d  = 0.5
const v  = 14
const iv = 1/v
const Arg     = v-K+1:v
const df_p1   = v+1                     # degrees of freedom from Wishart EIS initial sampler. See eq (17)
const inds    = find(tril!(ones(K,K)))
const Nb      = length(inds)            # number of estimated parameters from the covariance matrix or from the \Gamma matrix filled with (k^2+k)/2 auxiliary parameters
const ind_    = find(eye(K))            # index from a diagonal KxK matrix
const itmax   = 5;                      # maximum number of iterations;
const tol     = 0.01;                   # EIS tolerance to stop iterations;
const semente = 123456789
const Vini    = eye(K)./0.15


3×3 Array{Float64,2}:
 6.66667  0.0      0.0    
 0.0      6.66667  0.0    
 0.0      0.0      6.66667

In [2]:
invC = PDMat([0.0238 0.0057 0.0145; 0.0057 0.0239 0.0056; 0.0145 0.0056 0.0330])
C    = inv(invC)
V0   = Vini
par  = [0.5000   14.0000    7.6761   -1.0835   -3.1890    6.6010   -1.1202    5.5048]


1×8 Array{Float64,2}:
 0.5  14.0  7.6761  -1.0835  -3.189  6.601  -1.1202  5.5048

In [3]:
yt = readdlm("data3.csv",',')

240×3 Array{Float64,2}:
 -0.885021    0.00196412  -0.678012
  1.792      -0.407463     2.14042 
  0.472394    0.556765    -1.49857 
 -0.0384967  -0.505739     0.221759
  1.01197     0.602443     1.33512 
 -0.0215475  -1.1741       0.352574
 -1.12179     0.557364    -1.65632 
  1.33181     2.07583      2.78358 
  0.151719    0.328076     1.90633 
 -3.0598     -2.40975     -4.4827  
  0.604914    1.51072      1.30826 
 -0.426768   -2.44504      0.856805
 -0.0649287   0.825995    -1.25525 
  ⋮                                
  1.24187     1.10862      0.76207 
  1.6812     -0.383984     1.71112 
 -1.02625    -0.522054    -0.656608
 -3.31528     0.658987     1.05307 
 -0.455854    1.10838     -2.66947 
  0.0418779   0.591068    -1.51447 
  0.742213    2.64011     -1.39113 
  6.14713     0.229653     3.12921 
 -0.476619   -0.0843958   -0.548609
  1.60292     0.645093     1.76743 
 -1.36957     0.158419    -2.00862 
 -0.631276   -0.240438    -0.362757

In [4]:
function nearestSPD(A::Symmetric{Float64,Array{Float64,2}})
# nearestSPD - the nearest [in Frobenius norm()] Symmetric Positive Definite matrix to A
# usage: Ahat = nearestSPD[A]
#
# From Higham: "The nearest symmetric positive semidefinite matrix in the
# Frobenius norm to an arbitrary real matrix A is shown to be [B + H]/2
# where H is the symmetric polar factor of B=(A + A')/2."
#
# http://www.sciencedirect.com/science/article/pii/0024379588902236
#
# arguments: (input())
#  A - square matrix, which will be converted to the nearest Symmetric
#    Positive Definite Matrix.
#
# Arguments: (output)
#  Ahat - The matrix chosen as the nearest SPD matrix to A.


# symmetrize A into B
B = convert(Array,A)

# Compute the symmetric polar factor of B. Call it H.
# Clearly H is itself SPD.
U,Sigma,V = svd(B)
H = V*diagm(Sigma)*V'

# get Ahat in the above formula
Ahat = Symmetric((B+H)/2)

# ensure symmetry
# Ahat = (Ahat + Ahat')/2

# test that Ahat is in fact PD. if it is not so, then tweak it just a bit.
k = 0
p = 1
while p>0
    k = k+1
    try
        Ahat = PDMat(Ahat)
        p = -1
    catch
        # Ahat failed the chol test. It must have been just a hair off
        # due to floating point trash, so it is simplest now just to
        # tweak by adding a tiny multiple of an identity matrix.
        valor,Vec = eig(Ahat)
        mineig = minimum(valor)
        Ahat = Ahat + (-mineig*k.^2 + eps(mineig))*eye(size(A,1))
    end
end
return Ahat
end


nearestSPD (generic function with 1 method)

In [5]:
# Parameters and variables for EIS iterations
Neis    = 100
it      = 0
diff    = 100                         # initial difference;
bet     = zeros(Nb+1,T)              # initialize vector to save EIS parameters;
betas_0 = bet                         # initialize EIS parameters at zero for initial sampler. See paragraph bellow eq. (17);
VINI    = Array{Float64,3}(K,K,Neis)
for i=1:Neis
    VINI[:,:,i]    = Vini             # state at t=0. Initial condition is treated as given
end
Y       = Array{Float64,2}(Neis,T-1)    # dependent variable for EIS regression;
XX      = Array{Float64,3}(Neis,Nb+1,T) # set space for Neis draws of EIS parameters at each period t;
Ytm1    = Array{Float64,2}(Neis,1)      # set space for Neis evaluations of the EIS objective function.
indd    = Array{Int64,1}(K+1)
for i=1:K
    hu      = find(inds.==ind_[i])
    indd[i] = hu[1]                    # indicate the linear indice in "inds" which corresponds to a diagonal parameter;
end
indd[end] = Nb+1
cC        = zeros(K,K)
cC[inds]  = par[3:Nb+2]                # maps parameters to cC matrix in order to recriate matrix cC;
cC        = reshape(cC,K,K)            # reorganize matrix;
C         = PDMat(cC*cC')


PDMats.PDMat{Float64,Array{Float64,2}}(3, [58.9225 -8.31705 -24.4791; -8.31705 44.7472 -3.93916; -24.4791 -3.93916 41.7274], Base.LinAlg.Cholesky{Float64,Array{Float64,2}} with factor:
[7.6761 -1.0835 -3.189; 0.0 6.601 -1.1202; 0.0 0.0 5.5048])

In [6]:
cC

3×3 Array{Float64,2}:
  7.6761   0.0     0.0   
 -1.0835   6.601   0.0   
 -3.189   -1.1202  5.5048

In [10]:
# while it<itmax
t,i = 1,1
    # Use CRNs
    srand(semente)
    # Initialize EIS sampler. See eq.(17):
    V0        = Vini                     # initial state is given in this exercise;
    df_eis    = bet[end,:].+df_p1
    df_eis    = [df_eis; df_p1]          # degrees of freedom of EIS samplers from t=1 to t=T. Note that for t=T, df_eis=df_p1, since filter estimate is equal to smoothed estimate at T;
#     for t=1:T
        BetMat       = zeros(K,K)        # BetMat will be used as the \Gamma matrix from Eq. (17), which collects (k^2+k)/2 auxiliary parameters from the scale matrix of the EIS sampler
        BetMat[inds] = bet[1:Nb,t]
        BetMat       = BetMat+tril(BetMat,-1)'
        EISmat       = BetMat+yt[t,:]*yt[t,:]'
        df_eis_t     = df_eis[t];
#         if t==1
            cVd = V0.^(d/2)
            Veist = Array{Float64,3}(K,K,Neis)
            XXX   = Array{Float64,2}(Neis,Nb+1)
            sS  = C.chol[:L]*cVd*sqrt(iv)
            # Compute symmetric PDMatrix scale matrix
            S   = (sS*sS')
            EISmatS = EISmat*(sS*sS')
            Seis = PDMat(Symmetric(S-1/(1+trace(EISmatS))*S*EISmatS))
#             for  i=1:Neis
#                 Veist[:,:,i] = rand(Wishart(df_eis_t,Seis))
#                 XX[i,1,t] = Veist[1,1,i]*0.5
#                 XX[i,2,t] = Veist[2,1,i]
#                 XX[i,3,t] = Veist[3,1,i]
#                 XX[i,4,t] = Veist[2,2,i]*0.5
#                 XX[i,5,t] = Veist[3,2,i]
#                 XX[i,6,t] = Veist[3,3,i]*0.5
#                 XX[i,7,t] = logdet(Veist[:,:,i])*0.5
#             end
#             V0 = Veist
#         else
#             Veist = Array{Float64,3}(K,K,Neis)
#             XXX   = Array{Float64,2}(Neis,Nb+1)
#             for i=1:Neis
#                 # Compute Eigenvalue decomposition
#                 value,V = eig(V0[:,:,i])
#                 sS  = C.chol[:L]*V*diagm(value.^(d/2))*sqrt(iv)
#                 # Compute symmetric PDMatrix scale matrix
#                 S   = (sS*sS')
#                 EISmatS = EISmat*S
#                 Seis = PDMat(Symmetric(S-1/(1+trace(EISmatS))*S*EISmatS))
#                 # Compute integrating constant \Chi_{t} for smoothing. See Eq. (18).
#                 logdetStm1 = logdet(Seis)
#                 logdetS    = logdet(PDMat(S));
#                 Ytm1[i]    = 0.5*(df_eis_t*logdetStm1-v*logdetS); # log of integrating constant, which is a scalar;
#                 Veist[:,:,i] = rand(Wishart(df_eis_t,Seis))
#                 XX[i,1,t] = Veist[1,1,i]*0.5
#                 XX[i,2,t] = Veist[2,1,i]
#                 XX[i,3,t] = Veist[3,1,i]
#                 XX[i,4,t] = Veist[2,2,i]*0.5
#                 XX[i,5,t] = Veist[3,2,i]
#                 XX[i,6,t] = Veist[3,3,i]*0.5
#                 XX[i,7,t] = logdet(Veist[:,:,i])*0.5
#             end
#             Y[:,t-1] = Ytm1;
#             if t<T
#                 V0 = Veist
#             end
#         end
#     end
#     # Backward EIS loop to smooth EIS estimates of t (\Chi_{t}) with integrating constant of t+1 (\Chi_{t+1})
#     for t = T-1:-1:1
#         X = [ones(Neis,1) XX[:,:,t]]  # EIS regressors. Obs: X uses the sample of N draws to estime EIS parameters, collected by beta;
#         b = (X'*X)\(X'*Y[:,t])     # EIS parameters. Beta is a Nb+2 x 1 matrix;
#         bet[:,t] = b[2:end]        # Save EIS parameters eliminating those w.r.t. the constant. Matrix bet(:,t) becomes Nb+1 x 1 again;
#     end
#     # Check convergence:
#     diff = sum(sum(abs.(bet[1:end-1,:]-betas_0[1:end-1,:]))) # compute sum of individual relative difference between beta_{t} and beta_{t-1}. Obs: it is also possible to compute percentage change in parameters;
#     #println(diff)
#     betas_0 = copy(bet)
#     # Count EIS iterations
#     it = it+1
# end
# # Sample from optimized EIS sampler:
# V0   = Array{Float64,3}(K,K,N)
# for i=1:N
#     V0[:,:,i] = Vini              # state at t=0. Initial condition is treated as given
# end
# Veist  = Array{Float64,3}(K,K,N);
# df_eis = bet[end,:].+df_p1
# df_eis = [df_eis; df_p1]          # degrees of freedom of EIS samplers from t=1 to t=T. Note that for t=T, df_eis=df_p1, since filter estimate is equal to smoothed estimate at T;
# srand(semente*3)                  # use different seed after optimization

# # Evaluate EIS Integrand for MC integration using draws from optimized EIS sampler.
# ratio = Array{Float64,2}(T,N);
            
        


In [8]:
for t=1:T
#    ytt          = yt[t,:]
    BetMat       = zeros(K,K)              # BetMat will be used as the \Gamma matrix from Eq. (17), which collects (k^2+k)/2 auxiliary parameters from the scale matrix of the EIS sampler
    BetMat[inds] = bet[1:Nb,t]
    BetMat       = BetMat+tril(BetMat,-1)'
    EISmat       = BetMat+yt[t,:]*yt[t,:]'
    df_eis_t     = df_eis[t];
    ArgEIS       = (df_eis_t-K+1:df_eis_t)
    for i=1:N
        # Compute Eigenvalue decomposition
        value,V = eig(V0[:,:,i])
        sS  = C.chol[:L]*V*diagm(value.^(d/2))*sqrt(iv)
        # Compute symmetric PDMatrix scale matrix
        S   = (sS*sS')
        Ss  = PDMat(S)
        EISmatS = (Ss*EISmat)'
        Seis = Symmetric(S-1/(1+trace(EISmatS))*S*EISmatS)
        if isposdef(Seis)
            Seis = PDMat(Seis)
        else
            Seis = nearestSPD(Seis)
        end
        Veist[:,:,i] = rand(Wishart(df_eis_t,Seis))
        Veisti = PDMat(Veist[:,:,i])
        logdetVeis   = logdet(Veisti)
        # Measurement density in logs
        lgt = -K/2*LOG2PI+0.5*logdetVeis-0.5*quad(Veisti,yt[t,:])#yt[t,:]'*Veist[:,:,i]*yt[t,:]
        # State Transition Density in logs
        cnstt  = logdet(Ss)*(-v/2)-(v*K*.5)*LOG2-((K*(K-1))/4)*LOGPI-sum(lgamma.(Arg/2))
        kernel = logdetVeis*((v-K-1)/2) -.5*sum(diag(Ss\Veist[:,:,i]))
        lpt =  cnstt+kernel
        # Importance Sampler in logs
        cnstt  = logdet(Seis)*(-df_eis_t/2)-(df_eis_t*K*.5)*LOG2-((K*(K-1))/4)*LOGPI-sum(lgamma.(ArgEIS/2))
        kernel = logdetVeis*((df_eis_t-K-1)/2) -.5*sum(diag(Seis\Veist[:,:,i]))
        lmt    = cnstt+kernel
        # IS ratio
        ratio[t,i] = (lgt+lpt-lmt)
    end
    if t<T
        V0 = Veist
    end
end

# lik = mean(exp.(sum(ratio.-adj,1)))    # adjust ratio against overflows due to exponentiation;
# loglik = log(lik)+T*adj                # loglikelihood;
# n_loglik = -loglik

LoadError: DomainError:
Exponentiation yielding a complex result requires a complex argument.
Replace x^y with (x+0im)^y, Complex(x)^y, or similar.

In [70]:
BetMat       = zeros(K,K)              # BetMat will be used as the \Gamma matrix from Eq. (17), which collects (k^2+k)/2 auxiliary parameters from the scale matrix of the EIS sampler
    BetMat[inds] = bet[1:Nb,t]
    BetMat       = Symmetric(BetMat,:L)

3×3 Symmetric{Float64,Array{Float64,2}}:
 -0.145912   0.0170917   -0.114009 
  0.0170917  0.00982586   0.0392737
 -0.114009   0.0392737   -0.100044 

In [74]:
BetMat1       = rand(Wishart(1000,eye(3)))              # BetMat will be used as the \Gamma matrix from Eq. (17), which collects (k^2+k)/2 auxiliary parameters from the scale matrix of the EIS sampler
    BetMat1[inds] = bet[1:Nb,t]
@time    BetMat1       = Symmetric(BetMat1,:L)

  0.000005 seconds (5 allocations: 192 bytes)


3×3 Symmetric{Float64,Array{Float64,2}}:
 -0.145912   0.0170917   -0.114009 
  0.0170917  0.00982586   0.0392737
 -0.114009   0.0392737   -0.100044 

In [75]:
hu

3×3 Symmetric{Float64,Array{Float64,2}}:
 -0.145912   0.0170917   -0.114009 
  0.0170917  0.00982586   0.0392737
 -0.114009   0.0392737   -0.100044 

In [ ]:
# Sample from optimized EIS sampler:
V0   = Array{Float64,3}(K,K,N)
for i=1:N
    V0[:,:,i] = Vini              # state at t=0. Initial condition is treated as given
end
Veist  = Array{Float64,3}(K,K,N);
df_eis = bet[end,:].+df_p1 
df_eis = [df_eis; df_p1]          # degrees of freedom of EIS samplers from t=1 to t=T. Note that for t=T, df_eis=df_p1, since filter estimate is equal to smoothed estimate at T;
srand(semente*3)                  # use different seed after optimization

# Evaluate EIS Integrand for MC integration using draws from optimized EIS sampler.
ratio = Array{Float64,2}(T,N);

# Construct moments of EIS optimized sampler
for t=1:T
#    ytt          = yt[t,:]
    BetMat       = zeros(K,K)              # BetMat will be used as the \Gamma matrix from Eq. (17), which collects (k^2+k)/2 auxiliary parameters from the scale matrix of the EIS sampler
    BetMat[inds] = bet[1:Nb,t]
    BetMat       = BetMat+tril(BetMat,-1)'
    EISmat       = BetMat+yt[t,:]*yt[t,:]'
    df_eis_t     = df_eis[t];
    for i=1:N
        # Compute Eigenvalue decomposition
        value,V = eig(V0[:,:,i])
        cV0 = V*diagm(value).^(d/2)
        sS  = C.chol[:L]*cV0*(iv/2) 
        # Compute symmetric PDMatrix scale matrix
        S   = (sS*sS') 
#         Ss  = PDMat(S)
        EISmatS = EISmat*S
        Seis = PDMat(Symmetric(S-1/(1+trace(EISmatS))*S*EISmatS))
        Veist[:,:,i] = rand(Wishart(df_eis_t,Seis))
        Veisti = PDMat(Veist[:,:,i])
        logdetVeis   = logdet(Veisti)
        # Measurement density in logs
        lgt = -K/2*LOG2PI+0.5*logdetVeis-0.5*quad(Veisti,yt[t,:])#yt[t,:]'*Veist[:,:,i]*yt[t,:]
        # State Transition Density in logs
        cnstt  = logdet(S)*(-v/2)-(v*K*.5)*LOG2-((K*(K-1))/4)*LOGPI-sum(lgamma.(Arg/2))
        kernel = logdetVeis*((v-K-1)/2) -.5*sum(diag(S\Veist[:,:,i]))
        lpt =  cnstt+kernel
        # Importance Sampler in logs
        ArgEIS = (df_eis_t-K+1:df_eis_t)
        cnstt  = logdet(Seis)*(-df_eis_t/2)-(df_eis_t*K*.5)*LOG2-((K*(K-1))/4)*LOGPI-sum(lgamma.(ArgEIS/2))
        kernel = logdetVeis*((df_eis_t-K-1)/2) -.5*sum(diag(Seis\Veist[:,:,i]))
        lmt    = cnstt+kernel
        # IS ratio
        ratio[t,i] = (lgt+lpt-lmt) 
    end
    if t<T
        V0 = Veist
    end
end

In [ ]:
adj = -5.9
lik = mean(exp.(sum(ratio.-adj,1)))    # adjust ratio against overflows due to exponentiation;
loglik = log(lik)+T*adj                # loglikelihood;
n_loglik = -loglik      



In [ ]:
S

In [ ]:
hu=Veist[:,:,1]